In [1]:
import os

%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\End-to-End-MLOps-Project\\research'

In [2]:
os.chdir("../")

%pwd

'c:\\Users\\91889\\OneDrive\\Desktop\\End-to-End-MLOps-Project'

### To Integrate MLFlow on jupyter we will use the below code

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/CodeWithCharan/End-to-End-MLOps-Project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="CodeWithCharan"
os.environ["MLFLOW_TRACKING_PASSWORD"]="YourAccessToken"

### Config Entity

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    metric_file: str
    target_column: str
    all_params: dict
    mlflow_uri: str

### Configuration Manager

In [5]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            test_data_path = config.test_data_path,
            model_path = config.model_path,
            metric_file = config.metric_file,
            target_column = schema.name,
            all_params = params,
            mlflow_uri = "https://dagshub.com/CodeWithCharan/End-to-End-MLOps-Project.mlflow"
        )

        return model_evaluation_config

### Components - Model Evaluation

In [6]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [7]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metric(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            pred_qualities = model.predict(X_test)
            (rmse, mae, r2) = self.eval_metric(y_test, pred_qualities)

            # Save metrices to local json file
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric('rmse', rmse)
            mlflow.log_metric('mae', mae)
            mlflow.log_metric('r2', r2)

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticnetModel")
            else:
                mlflow.sklearn.log_model(model, "model")

### Pipeline

In [8]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()

except Exception as e:
    raise e

[2024-07-13 19:38:21,687: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-13 19:38:21,687: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-13 19:38:21,695: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-07-13 19:38:21,696: INFO: common: created directory at: artifacts]
[2024-07-13 19:38:21,699: INFO: common: created directory at: artifacts/model_evaluation]
[2024-07-13 19:38:22,816: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


c:\Users\91889\anaconda3\envs\mlopsenv\lib\site-packages\_distutils_hack\__init__.py:26: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'ElasticnetModel'.
2024/07/13 19:38:35 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticnetModel, version 1
Created version '1' of model 'ElasticnetModel'.
